In [1]:
import numpy as np 
from scipy.optimize import curve_fit
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from ipywidgets import interact
from thermo import Chemical, UNIFAC

In [2]:
conc = {'t': [0, 450, 850, 1256, 1664, 1969, 2343, 2751, 3159, 3600, 4381, 5026, 5875, 6928] , 
    'IB':  [1.85,1.519, 1.25, 1, 0.835, 0.675, 0.55, 0.47, 0.4, 0.345, 0.325, 0.325, 0.325, 0.325], 
    'EtOH': [1.8, 1.425, 1.175, 0.98, 0.79, 0.635, 0.5, 0.435, 0.350, 0.29, 0.275, 0.275, 0.275, 0.275], 
    'ETBE': [0, 0.325, 0.6, 0.81, 1, 1.15, 1.29, 1.375, 1.43, 1.475, 1.519, 1.519, 1.519, 1.519] }

def activity(T, P, Q, arr): # takes Fi (mol/s)
    x_ls = [C/sum(arr) for C in arr]
    UNIFAC_DG = fetch_UNIFAC_DG()
    return [g*x for g, x, in zip(UNIFAC(T, x_ls, UNIFAC_DG), x_ls)]

def fetch_UNIFAC_DG(full_name_list = ['isobutene', 'ethanol', 'ethyl tert-butyl ether'], update =False):
    UNIFAC_DG = []
    if update:
        for name in full_name_list:
            print(name)
            a = Chemical(name, 200, 100).UNIFAC_groups
            UNIFAC_DG.append(a)
        return UNIFAC_DG
    else:
        return [{1: 2, 7: 1}, {1: 1, 2: 1, 14: 1}, {1: 4, 4: 1, 25: 1}]

In [3]:
def Func(k =0.00047, Keq = 12.5872, B = 50, F= 7, D = 8):
    tspan = np.linspace(0, conc['t'][-1], 100)
    K1, K2 = 0,0
    def rate(arr):
        k2, Keq2 = k*20, Keq*20
        CIB, CEtOH, CETBE = activity(343, 101.325, 1, arr)
        return ((k*CIB*CEtOH - (k/Keq)*B*CETBE)/(CEtOH + B*CETBE + K1)) + ((k2*CIB*CEtOH - (k2/Keq2)*D*CETBE)/(CIB + F*CEtOH**2 + D*CETBE + K2))

    def batch(t, arr):
        r = rate(arr)
        return [-r, -r, r]
    y0 = [1.85, 1.8, 0]
    ans = solve_ivp(batch, [0, tspan[-1]], y0, dense_output=True).sol(tspan)
    plt.scatter(conc['t'], conc['IB'])
    plt.scatter(conc['t'], conc['EtOH'] )
    plt.scatter(conc['t'], conc['ETBE'])
    plt.plot(tspan, ans[0])
    plt.plot(tspan, ans[1])
    plt.plot(tspan, ans[2])
    plt.show()
    print('k ='+ str(k) + ', Keq = '+ str(Keq) + ', B = '+ str(B) + ', F= '+  str(F) +', D = ' +str(D))
#%%
interact(Func, k = (1e-6, 1, 1e-7), Keq = (1e-20, 20, 1e-5) , B=(0,50,10), F=(5,25,1), D=(0,20,2))

interactive(children=(FloatSlider(value=0.00047, description='k', max=1.0, min=1e-06, step=1e-07), FloatSlider…

<function __main__.Func(k=0.00047, Keq=12.5872, B=50, F=7, D=8)>